In [1]:
import asyncio
import aiohttp
from tqdm.asyncio import tqdm_asyncio
import pandas as pd
from pathlib import Path

from config_file import config

In [2]:
games_df = pd.read_csv(config.DATA_PATH / 'games.csv')

In [4]:
games_df.head(5)

,image_id,game_id,name,release_date,rating,genres,player_perspectives
0,scc9gr,1156,Tomb Raider II,880070400,80.270008,"['Shooter', 'Platform', 'Puzzle', 'Adventure']",['Third person']
1,scc9gs,1156,Tomb Raider II,880070400,80.270008,"['Shooter', 'Platform', 'Puzzle', 'Adventure']",['Third person']
2,scc9gt,1156,Tomb Raider II,880070400,80.270008,"['Shooter', 'Platform', 'Puzzle', 'Adventure']",['Third person']
3,scc9gq,1156,Tomb Raider II,880070400,80.270008,"['Shooter', 'Platform', 'Puzzle', 'Adventure']",['Third person']
4,scc9gp,1156,Tomb Raider II,880070400,80.270008,"['Shooter', 'Platform', 'Puzzle', 'Adventure']",['Third person']


In [6]:
import nest_asyncio
nest_asyncio.apply()

size = '720p'
IMAGE_DIR = config.IMAGES_PATH

async def download_image(session, image_id):
    url = f"https://images.igdb.com/igdb/image/upload/t_{size}/{image_id}.jpg"
    image_path = IMAGE_DIR / f"{image_id}.jpg"

    if image_path.exists():
        # print(f"✅ {image_id}.jpg уже скачан.")
        return

    try:
        async with session.get(url) as response:
            if response.status == 200:
                with open(image_path, "wb") as f:
                    f.write(await response.read())
                # print(f"✅ Скачано: {image_id}.jpg")
            else:
                pass
                # print(f"❌ Ошибка {response.status} при скачивании {image_id}")
    except Exception as e:
        print(f"⚠ Ошибка при скачивании {image_id}: {e}")

async def main(df: pd.DataFrame):
    async with aiohttp.ClientSession() as session:
        tasks = [download_image(session, image_id) for image_id in df["image_id"]]
        await tqdm_asyncio.gather(*tasks)

asyncio.run(main(games_df))

100%|██████████| 13/13 [00:00<00:00, 21.60it/s]
